In [1]:
from pathlib import Path
from data_import import (
    load_data, load_ecb_1y_yield,
    fill_liabilities, drop_high_leverage_firms,
    prepare_merton_inputs, prepare_nig_inputs
)

ret_daily, bs, coverage = load_data(
    Path.cwd() / "data/raw/Jan2025_Accenture_Dataset_ErasmusCase.xlsx",
    start_date="2012-01-01",
    end_date="2025-12-19",
    enforce_coverage=True,
    coverage_tol=0.95,
    liabilities_scale="auto",
    verbose=True,
)

df_rf = load_ecb_1y_yield(
    startPeriod="2010-01-01",
    endPeriod="2025-12-31",
    out_file="ecb_yc_1y_aaa.xml",
    verify_ssl=True,  # recommended if it works
)

df_cal = ret_daily[["date"]].drop_duplicates().sort_values("date").reset_index(drop=True)

debt_daily = fill_liabilities(bs, df_cal)

ret_filt, bs_filt, lev_by_firm, dropped = drop_high_leverage_firms(
    ret_daily,
    bs,
    df_calendar=df_cal,
    debt_daily=debt_daily,
    lev_threshold=8.0,
    lev_agg="median",
    verbose=True,
)

# keep debt panel consistent with filtered firms
keep = set(ret_filt["gvkey"].astype(str).unique())
debt_daily_filt = debt_daily[debt_daily["gvkey"].astype(str).isin(keep)].copy()

# Merton
merton_df = prepare_merton_inputs(ret_filt, bs_filt, df_rf, debt_daily=debt_daily_filt)

# NIG
nig_df, em_cache = prepare_nig_inputs(ret_filt, bs_filt, df_rf, debt_daily=debt_daily_filt, build_em=False)

[load_data] Firms (ret_daily): 46
[load_data] Date range (ret_daily): 2012-01-03 .. 2025-12-19
[load_data] Coverage min/median/max: 0.999 / 1.000 / 1.000
[load_data] liabilities_scale_used: 1e+06
[load_data] QA mcap_reported<=0 rows (raw windowed mkt): 62
Data has been written to ecb_yc_1y_aaa.xml
[drop_high_leverage_firms] agg=median, threshold=8.0
[drop_high_leverage_firms] firms before: 46 | after: 36
[drop_high_leverage_firms] dropped firms: 10


In [17]:
print(merton_df.head())
print(merton_df.info())
print(nig_df.head())
print(nig_df.info())


    gvkey       date             E  logret_mcap          isin  \
0  100022 2012-01-03  3.328431e+10     0.039286  DE0005190003   
1  100022 2012-01-04  3.363347e+10     0.010436  DE0005190003   
2  100022 2012-01-05  3.380203e+10     0.004999  DE0005190003   
3  100022 2012-01-06  3.344083e+10    -0.010743  DE0005190003   
4  100022 2012-01-09  3.421741e+10     0.022957  DE0005190003   

                        company country_iso         r             B  \
0  BAYERISCHE MOTOREN WERKE AKT         DEU  0.001177  8.576700e+10   
1  BAYERISCHE MOTOREN WERKE AKT         DEU  0.001037  8.576700e+10   
2  BAYERISCHE MOTOREN WERKE AKT         DEU  0.001614  8.576700e+10   
3  BAYERISCHE MOTOREN WERKE AKT         DEU  0.001873  8.576700e+10   
4  BAYERISCHE MOTOREN WERKE AKT         DEU  0.001835  8.576700e+10   

   sigma_E_daily  sigma_E  
0            NaN      NaN  
1            NaN      NaN  
2            NaN      NaN  
3            NaN      NaN  
4            NaN      NaN  
<class 'pandas